In [1]:
#get urls
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import re

#collecting all html pages
# from urllib.request import urlopen, Request
# from bs4 import BeautifulSoup
import re
import os

#get tsv files
# from urllib.request import urlopen, Request
# from bs4 import BeautifulSoup
# import re
# import os
import csv
from langdetect import detect
from progress.bar import IncrementalBar

In [2]:
#pre-process all the information
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from unidecode import unidecode
#import re

#Create your index! - vocabularyGenerator
#import csv
import json
from progress.counter import Counter
#import os

from prettytable import PrettyTable
import textwrap

import math
import heapq

In [3]:
import q1
import q2
import q5

# 1. Data collection

### 1.1. Get the list of books

In [5]:
#q1.get_urls()

### 1.2. Crawl books

In [6]:
#q1.collect_html_pages()

### 1.3 Parse downloaded pages

In [7]:
#q1.get_all_tsv()

# 2. Search Engine

In [4]:
##pre-process all the information
q2.download()

[nltk_data] Downloading package stopwords to C:\Users\Simone-
[nltk_data]     MSI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Simone-
[nltk_data]     MSI\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 2.1. Conjunctive query

#### 2.1.1 Create your index!

##### Create vocabulary

In [5]:
vocabulary = dict()

In [48]:
bar = Counter('Progress: ')

for i in range(1, 301):
    path = f'TSV_books/list_page_{i}/'
    
    for file in os.scandir(path):
        with open(path + file.name) as tsv_file:
            try:
                tsv_reader = csv.DictReader(tsv_file, delimiter='\t')
                row = tsv_reader.__next__()['plot']
            
                for word in q2.preprocessor(row):
                    if word not in vocabulary.keys():
                        vocabulary[word] = len(vocabulary)
            except:
                #print(f'{path + file.name}')
                continue
        bar.next()
bar.finish()

with open('generated/vocabulary.json', 'w') as json_file:
    json.dump(vocabulary, json_file)

print('File generated.')

File generated.


##### create invertedIndex

In [52]:
invertedIndex = dict()

with open('generated/vocabulary.json', 'r') as json_file:
    vocabulary = json.load(json_file)

for word in vocabulary:
    invertedIndex[vocabulary[word]] = []

bar = Counter('Progress: ')
for i in range(1, 301):
    path = f'TSV_books/list_page_{i}/'
    
    for file in os.scandir(path):
        with open(path + file.name) as tsv_file:
            documentId = re.sub(r'(article\_|\.tsv)', '', file.name)
            tsv_reader = csv.DictReader(tsv_file, delimiter='\t')
            try:
                row = tsv_reader.__next__()['plot']

                for word in q2.preprocessor(row):
                    if documentId not in invertedIndex[vocabulary[word]]:
                        invertedIndex[vocabulary[word]].append(documentId)
            except:
                #print(f'{path + file.name}')
                continue
        bar.next()
bar.finish()


with open('generated/invertedIndex.json', 'w') as json_file:
    json.dump(invertedIndex, json_file)

print('File generated.')

File generated.


#### 2.1.2 Execute the query

In [8]:
query = input('Query: ')
query = q2.preprocessor(query)
resultId = q2.search(query)

for i in resultId:
    with open(f'TSV_books/list_page_{i // 100 + 1}/article_{i}.tsv') as tsv_file:
        tsv_reader = csv.DictReader(tsv_file, delimiter='\t')
        row = tsv_reader.__next__()

    with open('generated/url_collected.txt') as urls:
        for j, url in enumerate(urls):
            if j == i:
                print(f"Title: {row['bookTitle']}\nUrl: {url}\nPlot:\n{row['plot']}")
                print('--------------------------------------------------')

Query: survival games
Title: The Hunger Games
Url: https://www.goodreads.com/book/show/2767052-the-hunger-games

Plot:
Could you survive on your own in the wild, with every one out to make sure you don't live to see the morning? In the ruins of a place once known as North America lies the nation of Panem, a shining Capitol surrounded by twelve outlying districts. The Capitol is harsh and cruel and keeps the districts in line by forcing them all to send one boy and one girl between the ages Could you survive on your own in the wild, with every one out to make sure you don't live to see the morning? In the ruins of a place once known as North America lies the nation of Panem, a shining Capitol surrounded by twelve outlying districts. The Capitol is harsh and cruel and keeps the districts in line by forcing them all to send one boy and one girl between the ages of twelve and eighteen to participate in the annual Hunger Games, a fight to the death on live TV. Sixteen-year-old Katniss Everd

Title: Skin Game
Url: https://www.goodreads.com/book/show/19486421-skin-game

Plot:
Harry Dresden, Chicago's only professional wizard, is about to have a very bad dayâ€¦ Because as Winter Knight to the Queen of Air and Darkness, Harry never knows what the scheming Mab might want him to do. Usually, itâ€™s something awful. He doesnâ€™t know the half of itâ€¦ Mab has just traded Harryâ€™s skills to pay off one of her debts. And now he must help a group of supernatural v Harry Dresden, Chicago's only professional wizard, is about to have a very bad dayâ€¦ Because as Winter Knight to the Queen of Air and Darkness, Harry never knows what the scheming Mab might want him to do. Usually, itâ€™s something awful. He doesnâ€™t know the half of itâ€¦ Mab has just traded Harryâ€™s skills to pay off one of her debts. And now he must help a group of supernatural villainsâ€”led by one of Harryâ€™s most dreaded and despised enemies, Nicodemus Archleoneâ€”to break into the highest-security vault in town

Title: Glow
Url: https://www.goodreads.com/book/show/7857713-wolfsbane

Plot:
One of School Library Journal â€™s Best Fiction Books of 2011 If a violent battle destroyed the only world youâ€™ve ever known, would you be brave enough to save who was left? Would love be strong enough to survive the fight? Either way, thereâ€™s no turning back. The Empyrean is the only home 15-year-old Waverly has ever known. Part of the first generation to be successfully conce One of School Library Journal â€™s Best Fiction Books of 2011 If a violent battle destroyed the only world youâ€™ve ever known, would you be brave enough to save who was left? Would love be strong enough to survive the fight? Either way, thereâ€™s no turning back. The Empyrean is the only home 15-year-old Waverly has ever known. Part of the first generation to be successfully conceived in deep space, she and her boyfriend Kieran will be pioneers ofÂ New Earth.Â Waverly knows she must marry young in order to have children who can ca

Title: The End of The Computer
Url: https://www.goodreads.com/book/show/23400620-odes

Plot:
Travel to the edge of The End of the world with some of the most interesting and enjoyable people you'll ever meet. Share the spine-chilling thrills and great emotional epiphanies as this company of friends are forced into the adventure of leaving their hollow lives in Moscow and Berkeley and becoming the earth mothers and fathers of Thunder Valley... if any of them survive Travel to the edge of The End of the world with some of the most interesting and enjoyable people you'll ever meet. Share the spine-chilling thrills and great emotional epiphanies as this company of friends are forced into the adventure of leaving their hollow lives in Moscow and Berkeley and becoming the earth mothers and fathers of Thunder Valley... if any of them survive the journey. Thrill with the discovery that a hypercomputer can be fun, kind, happy... and the most dangerous person in existence to those who would dest

Title: Blood Moon
Url: https://www.goodreads.com/book/show/13650334-legacy-of-a-dreamer

Plot:
Guilty until you survive all zonesâ€¦The Orcus Games is ready to put you on trialâ€¦ The lives of a witch and wizard couple, a new vampire and a rogue lycan intertwine in three magical tales of love, loss and revenge. The Craft - When witch and wizard couple, Heather and Phoenix make that fatal mistake on that, â€˜Blood Moonâ€™ night, they never thought they would be forced to worki Guilty until you survive all zonesâ€¦The Orcus Games is ready to put you on trialâ€¦ The lives of a witch and wizard couple, a new vampire and a rogue lycan intertwine in three magical tales of love, loss and revenge. The Craft - When witch and wizard couple, Heather and Phoenix make that fatal mistake on that, â€˜Blood Moonâ€™ night, they never thought they would be forced to working with a loner werewolf butâ€¦is he the only one who saw the, â€˜accidentalâ€™ fatal act? The Clan â€“ Ardan would do anything for hi

### 2.2 Conjunctive query & Ranking score

#### 2.2.1 Inverted index

In [6]:
invertedIndexTfidf = dict()
documentNorm = dict()

with open('generated/vocabulary.json', 'r') as json_file:
    vocabulary = json.load(json_file)

with open('generated/invertedIndex.json', 'r') as json_file:
    invertedIndex = json.load(json_file)

for word in vocabulary:
    invertedIndexTfidf[vocabulary[word]] = []

n = 0
for i in range(1, 301):
    path = f'TSV_books/list_page_{i}/'
    n += len([1 for _ in os.scandir(path)])

bar = Counter('Progress: ')
for i in range(1, 301):
    path = f'TSV_books/list_page_{i}/'

    for file in os.scandir(path):
        norm = []
        with open(path + file.name) as tsv_file:
            documentId = re.sub(r'(article\_|\.tsv)', '', file.name)
            tsv_reader = csv.DictReader(tsv_file, delimiter='\t')
            try:
                plot = tsv_reader.__next__()['plot']
            except:
                #print(f'{path + file.name}')
                continue
                
            words = q2.preprocessor(plot)
            doneWords = []
            for word in words:
                if word not in doneWords:
                    tf = len([w for w in words if w == word])
                    N = len(invertedIndex[str(vocabulary[word])])
                    idf = math.log(n/N)
                    invertedIndexTfidf[vocabulary[word]].append((int(documentId), tf * idf))
                    doneWords.append(word)

                norm.append(invertedIndexTfidf[vocabulary[word]][-1][1])

                
        documentNorm[documentId] = math.sqrt(sum([x**2 for x in norm]))

        bar.next()
bar.finish()


with open('generated/invertedIndexTfidf.json', 'w') as json_file:
    json.dump(invertedIndexTfidf, json_file)

with open('generated/documentNorm.json', 'w') as json_file:
    json.dump(documentNorm, json_file)

print('File generated.')

File generated.


#### 2.2.2 Execute the query

In [7]:
query = input('Query: ')
query = q2.preprocessor(query)
resultId = q2.search(query)
k = 10

with open('generated/documentNorm.json', 'r') as json_file:
    documentNorm = json.load(json_file)

with open('generated/invertedIndexTfidf.json', 'r') as json_file:
    invertedIndexTfidf = json.load(json_file)
    
with open('generated/vocabulary.json', 'r') as json_file:
    vocabulary = json.load(json_file)

heapResult = []
heapq.heapify(heapResult)
for result in resultId:
    norm = documentNorm[str(result)]

    summation = 0
    for token in query:
        if token in vocabulary:
            summation += [x for x in invertedIndexTfidf[str(vocabulary[token])] if x[0] == int(result)][0][1]

    similarity = summation / norm

    heapq.heappush(heapResult, (similarity, result))

for i in heapq.nlargest(k, heapResult):
    with open(f'TSV_books/list_page_{i[1] // 100 + 1}/article_{i[1]}.tsv') as tsv_file:
        tsv_reader = csv.DictReader(tsv_file, delimiter='\t')
        row = tsv_reader.__next__()

    with open('generated/url_collected.txt') as urls:
        for j, url in enumerate(urls):
            if j == i[1]:
                print(f"Title: {row['bookTitle']}\nUrl: {url}\nSimilarity: {i[0]}\nPlot:\n{row['plot']}")
                print('--------------------------------------------------')

Query: survival games
Title: The Warden
Url: https://www.goodreads.com/book/show/32991547-natal-fault

Similarity: 0.44480051070972965
Plot:
Alice has led a normal life up until now. She wakes up finding herself trapped in a sick game of survival within the walls of an old asylum. She has to fight her way out while facing psychotic enemies, no one said they were all living psychopaths though... When she discovers how she got there she is forced to make difficult decisions. Will Alice survive the horror games?
--------------------------------------------------
Title: Devil's Own
Url: https://www.goodreads.com/book/show/1198929.Mountain_Solo

Similarity: 0.39352965240489546
Plot:
After surviving slavery, Aiden MacAlpin has nothing but thoughts of vengeance. When his tutor Elspeth learns a secret to his past, it thrusts them both into a game of passion and deception that neither may survive.
--------------------------------------------------
Title: U-10
Url: https://www.goodreads.com/book

# 3. Define a new score!

In [9]:
query = input('Query:')

query = query.lower()
pages = re.findall(r'(\d+)( |)page(s|)', query)
if len(pages) > 0:
    pages = pages[0][0]
else:
    pages = None
query = re.sub(r'\d+( |)page(s|)', '', query)
rating = re.findall(r'(\d+)( |)star(s|)', query)
if len(rating) > 0:
    rating = rating[0][0]
else:
    rating = None
query = re.sub(r'\d+( |)star(s|)', '', query)
year = re.findall(r'(^|\s)(\d{4})(\s|$)', query)
if len(year) > 0:
    year = year[0][1]
else:
    year = None
details = re.findall(r'details: (.*)$', query)
if len(details) > 0:
    details = details[0][0]
else:
    details = None
query = re.sub(r'details: (.*)$', '', query)

query = q2.preprocessor(query)
resultId = q2.search(query)
k = 10

with open('generated/documentNorm.json', 'r') as json_file:
    documentNorm = json.load(json_file)

with open('generated/invertedIndexTfidf.json', 'r') as json_file:
    invertedIndexTfidf = json.load(json_file)
    
with open('generated/vocabulary.json', 'r') as json_file:
    vocabulary = json.load(json_file)

plotSimilarityList = []
infoScoreList = []
pageScoreList = []
yearScoreList = []
ratingScoreList = []
for result in resultId:
    norm = documentNorm[str(result)]

    summation = 0
    for token in query:
        if token in vocabulary:
            summation += [x for x in invertedIndexTfidf[str(vocabulary[token])] if x[0] == int(result)][0][1]

    plotSimilarityList.append(summation / norm)

    with open(f'TSV_books/list_page_{result // 100 + 1}/article_{result}.tsv') as tsv_file:
        tsv_reader = csv.DictReader(tsv_file, delimiter='\t')
        row = tsv_reader.__next__()

    if details != None:
        authorScore = len([x for x in preprocessor(details) if x in preprocessor(row['bookAuthors'])])
        charactersScore = len([x for x in preprocessor(details) if x in preprocessor(row['characters'])])
        settingScore = len([x for x in preprocessor(details) if x in preprocessor(row['setting'])])
        infoScoreList.append(authorScore + charactersScore + settingScore)
    else:
        infoScoreList.append(0)

    if pages != None and row['numberOfPages'] != '':
        pageScoreList.append(1 / (abs(int(pages) - int(row['numberOfPages'])) + 1))
    else:
        pageScoreList.append(0)

    if year != None and row['publishingDate'] != '':
        if re.search(r'\d{4}', row['publishingDate']):
            yearScoreList.append(1 / (abs(int(year) - int(re.search(r'\d{4}', row['publishingDate']).group()) + 1)))
        else:
            yearScoreList.append(0)
    else:
        yearScoreList.append(0)

    if rating != None and row['ratingValue'] != '':
        ratingScoreList.append(1 / (abs(int(rating) - float(row['ratingValue'])) + 1))
    else:
        ratingScoreList.append(0)

infoScoreList = [x / (max(infoScoreList) + 1e-6) for x in infoScoreList]
pageScoreList = [x / (max(pageScoreList) + 1e-6) for x in pageScoreList]
yearScoreList = [x / (max(yearScoreList) + 1e-6) for x in yearScoreList]
ratingScoreList = [x / (max(ratingScoreList) + 1e-6) for x in ratingScoreList]

heapResult = []
heapq.heapify(heapResult)
for i, result in enumerate(resultId):
    similarity = (plotSimilarityList[i] + infoScoreList[i] + pageScoreList[i] + yearScoreList[i] + ratingScoreList[i]) / 5
    heapq.heappush(heapResult, (similarity, result))

for i in heapq.nlargest(k, heapResult):
    with open(f'TSV_books/list_page_{i[1] // 100 + 1}/article_{i[1]}.tsv') as tsv_file:
        tsv_reader = csv.DictReader(tsv_file, delimiter='\t')
        row = tsv_reader.__next__()

    with open('generated/url_collected.txt') as urls:
        for j, url in enumerate(urls):
            if j == i[1]:
                print(f"Title: {row['bookTitle']}\nUrl: {url}\nSimilarity: {i[0]}\nPlot:\n{row['plot']}")
                print('--------------------------------------------------')

Query:survival games
Title: The Warden
Url: https://www.goodreads.com/book/show/32991547-natal-fault

Similarity: 0.08896010214194593
Plot:
Alice has led a normal life up until now. She wakes up finding herself trapped in a sick game of survival within the walls of an old asylum. She has to fight her way out while facing psychotic enemies, no one said they were all living psychopaths though... When she discovers how she got there she is forced to make difficult decisions. Will Alice survive the horror games?
--------------------------------------------------
Title: Devil's Own
Url: https://www.goodreads.com/book/show/1198929.Mountain_Solo

Similarity: 0.0787059304809791
Plot:
After surviving slavery, Aiden MacAlpin has nothing but thoughts of vengeance. When his tutor Elspeth learns a secret to his past, it thrusts them both into a game of passion and deception that neither may survive.
--------------------------------------------------
Title: U-10
Url: https://www.goodreads.com/book/s

# 5. Algorithmic Question

In [6]:
s = "CADFECEILGJHABNOPSTIRYOEABILCNR."

In [8]:
subseq = q5.subsequence(s)

In [9]:
%%time
max(subseq, key=len)

Wall time: 997 µs


'CDFILNOPSTY'

In [10]:
ex1 = "bstshtbsrjyrsjhaezrhfhatrtmsmyujkjr6jnhnsfnsfyjryj"

In [12]:
subs = q5.subsequence(ex1)

In [13]:
%%time
max(subs, key=len)

Wall time: 997 µs


'befhjkrsy'

In [15]:
ex2 = "dcjskcnsiaomcncjsoansjcidnsfsubfvoaienovaiuenfaclweiuryogaiwutgvoaiuweygfoawugeroawievrawieurgvoawirteaoierbfabscn"

In [17]:
%%time
subs = q5.subsequence(ex2)
max(subs, key=len)

Wall time: 4.84 s


'acdfinoruvwy'

In [18]:
ex3 = "hfejffegoaienovaiuenfaclweiuryogaiwutgvoaiuweygfoawugeroawievrawieurgvoawirteaoierbfabs\
        aurfbvaliuebvluerbvaoliutervbaiuewtbvaiuytebaowybrvla8yrbvaowbvwuebvawubvoaiewbvlwurbg"

In [20]:
%%time
subs = q5.subsequence(ex3)
max(subs, key=len)

Wall time: 3min 21s


'acefgiorstuvwy'

In [21]:
s1 = "CA"
s2 = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

In [ ]:
q5.long_com_seq(s1, s2)

In [ ]:
s1 = "NCSDNDNNCKSMNVDKSCMDSKKCMD"
s2 = "CNNVNDJFNDNVND"

In [ ]:
long_com_seq(s1, s2)

The runtime complexity is given by the length of the input strings. If we call them *m* and *n*, the complexity is ***O(mn)***.

When *n* is the length of the alphabet, since *n* = 24 is a constant, the running time will be given only by *m* and therefore: ***O(m)***

##### BONUS

base case: we show that the formula is verified for $i=1, X[1]$
<br>
$X[1]= 1$, if there does not exists such a $j$
inductive step: let us now assume that the formula in question is verified for $i$
<br>
$X[i] = 1 + max\{X[j]; j = 0, ..., i-1, such\space{  }that\space{  }S[j]<S[i]\}$
we prove the validity of the formula for $i+1$
<br>
$X[i+1] = X[i] + 1$
<br>
$X[i+1] = 1 + 1 + max\{X[j]; j = 0, ..., i-1=j, such\space{  }that\space{  }S[j]<S[i]\}$
<br>
if we assume a certain j such that we obtain
<br>
$X[i+1] = 1 + max\{X[j]; j = 0, ..., i=j, such\space{  }that\space{  }S[j]<S[i+1]\}$
<br>
the formula is proved